Simple idea:

    - We can partition our code / functionality into two streams:
    i) Finding whales / profitable wallets etc.
    ii) Mapping these wallets to something.

Example of i) we already have written some `early_whales` functionality. For ii) we could use `intersect_dict` functionality. e.g. after identifying several wallets of interest (through whatever means) once per day we could run some code to compute the current intersection of their portfolios. How this might play out: we identify 20 wallets that are highly profitable. Some were early pepe whales, for example. Then, once per day, we run some code to compute the intersection of their portfolios. We notice that 8/20 have purchased a coin $\theta$. Maybe $\theta$ is a newly launched coin - this would be of particular interest! Maybe $\theta$ is a "trending" token. And so on.

Anyway, we basically have the functionality to do this (once given the wallets of interest - which we already have a few of).

In [1]:
from api_endpoints.etherscan_api import *
from api_endpoints.covalent_api import *
from etherscan import Etherscan #externally installed package https://github.com/pcko1/etherscan-python

from fastcore.test import *
from fastcore.basics import *

Alright. Let's try a slightly different track to the early whales track. How about we do the following:

1) Compute the current top_n holders of several tokens (meaning, get their portfolios at present).
2) Compute the intersect_dict

Then we can try several thresholds, and see if there are coins held by several of the accounts. Ok, cool.

In [9]:
import time
#Step 1: Given several address, get the top n token holders for each address.

###setup
cov_api = Covalent_Api()
chainName='eth-mainnet'
n=10

###

#Addresses: pepe, hpbitcoin, unibot
#pepe_address (already have: imported from etherscan_api.py)
hpbitcoin_address = '0x72e4f9f808c49a2a61de9c5896298920dc4eeea9' #The og harrypotterbitcoin address
unibot_address = '0xf819d9cb1c2a819fd991781a822de3ca8607c3c9'

addresses = [pepe_address,hpbitcoin_address,unibot_address]
token_holders_dict = {address:[] for address in addresses}

for _address in addresses:

    _token_holders = cov_api.get_token_holders(chainName=chainName,tokenAddress=_address,page_size=100,page_number=0)
    _token_holders = _token_holders[0:n]
    token_holders_dict[_address] = _token_holders
    print(f'got address {_address}')
    time.sleep(5)

#top n holders for each address. so shoulder have length n*len(addresses).
token_holders_list = [_holder for address in addresses for _holder in token_holders_dict[address]]

test_eq(len(token_holders_list),n*len(addresses))

RequestException occurred: HTTPSConnectionPool(host='api.covalenthq.com', port=443): Read timed out. (read timeout=10)


TypeError: 'NoneType' object is not subscriptable

In [3]:
#Step 2: Given the top n token holders, get their portfolios, and intersect them. All this can be done through a single call:

date = '2023-09-28' #todays date
token_holders_data = WalletListToIntersectDict(cov_api=cov_api,chainName=chainName,date=date,wallet_list=token_holders_list)


In [8]:
contract_name_if_more_than_k_holders(intersect_dict=token_holders_data.intersect_dict,k=5)

[('HarryPotterObamaSonic10Inu', '0x72e4f9f808c49a2a61de9c5896298920dc4eeea9'),
 ('Ether', '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee'),
 ('Unibot', '0xf819d9cb1c2a819fd991781a822de3ca8607c3c9'),
 ('HarryPotterObamaPacMan8Inu', '0x07e0edf8ce600fb51d44f51e3348d77d67f298ae'),
 ('Wrapped Ether', '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2')]

In [5]:
len(token_holders_list)

3

In [18]:
dextools_unibot_address = '0xf819d9cb1c2a819fd991781a822de3ca8607c3c9'

In [19]:
dextools_unibot_address == unibot_address

True

In [16]:
unibot_address = '0xf819d9cb1c2a819fd991781a822de3ca8607c3c9'
_token_holders = cov_api.get_token_holders(chainName=chainName,tokenAddress=unibot_address,page_size=100,page_number=0)


In [17]:
_token_holders

[{'contract_decimals': 18,
  'contract_name': 'Unibot',
  'contract_ticker_symbol': 'UNIBOT',
  'contract_address': '0xf819d9cb1c2a819fd991781a822de3ca8607c3c9',
  'supports_erc': ['erc20'],
  'logo_url': 'https://logos.covalenthq.com/tokens/1/0xf819d9cb1c2a819fd991781a822de3ca8607c3c9.png',
  'address': '0x8dbee21e8586ee356130074aaa789c33159921ca',
  'balance': '42446578658182434087300',
  'total_supply': '1000000000000000000000000',
  'block_height': 18233829},
 {'contract_decimals': 18,
  'contract_name': 'Unibot',
  'contract_ticker_symbol': 'UNIBOT',
  'contract_address': '0xf819d9cb1c2a819fd991781a822de3ca8607c3c9',
  'supports_erc': ['erc20'],
  'logo_url': 'https://logos.covalenthq.com/tokens/1/0xf819d9cb1c2a819fd991781a822de3ca8607c3c9.png',
  'address': '0xaee2ae13ebf81d38df5a9ed7013e80ea3f72e39b',
  'balance': '26168476930128468328603',
  'total_supply': '1000000000000000000000000',
  'block_height': 18233829},
 {'contract_decimals': 18,
  'contract_name': 'Unibot',
  'contr

In [14]:
print(_token_holders)

None


In [6]:

token_holders_pepe = cov_api.get_token_holders(chainName=chainName,tokenAddress=pepe_address,page_size=100,page_number=0)

type(token_holders_pepe)


list

In [7]:
len(token_holders_pepe)

100

len(token_holders_pepe)